## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [56]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [57]:
import requests
import pandas as pd
import json
from datetime import datetime

## Download

Retrieve the page

In [58]:
url = "https://services1.arcgis.com/P5Mv5GY5S66M8Z1Q/arcgis/rest/services/NCOV_ZipCode_Data_Public/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [59]:
r = requests.get(url)

In [60]:
data = r.json()

## Parse

In [61]:
dict_list = []

In [62]:
for item in data["features"]:
    d = dict(
        county="Sonoma",
        zip_code=item["attributes"]["ZipCode"],
        city=item["attributes"]["Community"],
        confirmed_cases=item["attributes"]["TotalCases"],
    )
    dict_list.append(d)

In [63]:
df = pd.DataFrame(dict_list)

In [64]:
df["city"] = df["city"].str.title()

In [65]:
df["area"] = df["zip_code"].astype(str) + ": " + df["city"]

Get timestamp

In [66]:
date_url = "https://services1.arcgis.com/P5Mv5GY5S66M8Z1Q/arcgis/rest/services/NCOV_ZipCode_Data_Public/FeatureServer/0/?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [67]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [68]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [69]:
latest_date = pd.to_datetime(timestamp).date()

In [70]:
df["county_date"] = latest_date

Filter out place with no cases

In [71]:
export_df = df[~df.confirmed_cases.isnull()].rename(columns={"zip_code": "zip"})[
    ["county", "area", "confirmed_cases", "county_date", "zip"]
]

In [72]:
try:
    assert not len(export_df) > 32
except AssertionError:
    raise AssertionError("Sonoma County's scraper has extra rows")

In [73]:
try:
    assert not len(export_df) < 32
except AssertionError:
    raise AssertionError("Sonoma County's scraper is missing rows")

## Export

Set date

In [74]:
tz = pytz.timezone("America/Los_Angeles")

In [75]:
today = datetime.now(tz).date()

In [76]:
slug = "sonoma"

In [77]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [78]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [79]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [80]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17088 entries, 8384 to 31
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   county           17088 non-null  object        
 1   area             17088 non-null  object        
 2   confirmed_cases  17088 non-null  object        
 3   county_date      8672 non-null   datetime64[ns]
 4   zip              8672 non-null   float64       
 5   date             17088 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 934.5+ KB


In [81]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)